In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms

from copy import deepcopy

In [ ]:
config = {
    "EPOCH": 300,
    "BATCH_SIZE": 50,
    "VALIDATION_BATCH_SIZE": 500,
    "DEVICE": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
    "AUGMENTATION_PROB": 0.5,
    "NORM_LAYER": "", # batch normalization
    # "NORM_LAYER": "gn", # group normalization
}

In [ ]:
def listToString(l): 
    return " ".join(str(l))

def printJSON(alpha, acc, net, step = None):
    """Create the json artifacts file

    Parameters
    ----------
    alpha: float
        value of alpha
    acc: list
        list of accuracies at different iterations
    net: the actual network configuration
    step: int
        current value of iteration
    """
    artifacts_dir = "artifacts"

    artifact_filename = f"ALPHA_{alpha}"
    if step is not None:
      artifact_filename += f"_STEPS_{step}"
    
    if config["AUGMENTATION_PROB"] > 0:
      artifact_filename += f"_T"

    artifact_filename += f"_{config['NORM_LAYER'].upper()}" if config['NORM_LAYER'] else ""
      
    # Parameters of the trained model
    server_model = net.state_dict()
    # Save the model on the local file system
    torch.save(server_model, f"{artifacts_dir}/{artifact_filename}.pth")
    config_copy = deepcopy(config)
    #config_copy["DIRICHELET_ALPHA"] = listToString(config_copy["DIRICHELET_ALPHA"])
    #config_copy["AVERAGE_ACCURACY"] = np.array2string(config_copy["AVERAGE_ACCURACY"])
    config_copy["DEVICE"] = ""
    data = {
        "config": config_copy,
        "accuracy": acc
    }

    with open(f"{artifacts_dir}/{artifact_filename}.json", "w") as f:
        f.write(json.dumps(data, indent=4))

In [ ]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv1 = nn.Conv2d(3, 64, 5)
        if config["NORM_LAYER"] == "bn":
            self.norm1 = nn.BatchNorm2d(64)
        elif config["NORM_LAYER"] == "gn":
            self.norm1 = nn.GroupNorm(4, 64)
        
        self.conv2 = nn.Conv2d(64, 64, 5)
        if config["NORM_LAYER"] == "bn":
            self.norm2 = nn.BatchNorm2d(64)
        elif config["NORM_LAYER"] == "gn":
            self.norm2 = nn.GroupNorm(4, 64)
        
        self.fc1 = nn.Linear(64 * 5 * 5, 384)
        self.fc2 = nn.Linear(384, 192)
        self.fc3 = nn.Linear(192, 10)

    def forward(self, x):
        if config["NORM_LAYER"] in ['bn', 'gn']:
            x = F.max_pool2d(F.relu(self.norm1(self.conv1(x))), (2,2))
            x = F.max_pool2d(F.relu(self.norm2(self.conv2(x))), 2)
        else:
            x = F.max_pool2d(F.relu(self.conv1(x)), (2,2))
            x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        
        x = torch.flatten(x, 1)  # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()
net = net.to(config["DEVICE"])

# print the total number of parameters
# from functools import reduce
# print(sum([reduce(lambda a, b: a * b, p.size()) for p in net.parameters()])) # 797962

In [ ]:
random_transform = transforms.Compose(
    [
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(1),
        transforms.ColorJitter(0.9, 0.9)
    ]
)

train_set = torchvision.datasets.CIFAR10(
    root="./data",
    train=True,
    download=True,
    transform=transforms.Compose(
        [
            transforms.ToTensor(),
            transforms.RandomApply([random_transform], config["AUGMENTATION_PROB"]),
            transforms.Normalize(mean=[0.491, 0.482, 0.447], std=[0.247, 0.243, 0.262]), # from the net, there are the values of cifer10
        ]
    ),
)

train_loader = torch.utils.data.DataLoader(
    train_set, batch_size=config["BATCH_SIZE"], shuffle=True, num_workers=2
)

test_set = torchvision.datasets.CIFAR10(
    root="./data",
    train=False,
    download=True,
    transform=transforms.Compose(
        [
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.491, 0.482, 0.447], std=[0.247, 0.243, 0.262]),
        ]
    ),
)

test_loader = torch.utils.data.DataLoader(
    test_set, batch_size=config["VALIDATION_BATCH_SIZE"], shuffle=False, num_workers=2
)

In [ ]:
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau

criterion = nn.CrossEntropyLoss(reduction='sum') # i sum the loss sample and i divide by the # of samples so i can have the test loss
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9, weight_decay=1e-3)

scheduler = ReduceLROnPlateau(optimizer, "min", factor=0.5, min_lr=1e-6, verbose=True)

In [ ]:
def compute_loss_accuracy(net, dataloader):
    correct = 0
    net.train(False)

    with torch.no_grad():
        loss, n = 0, 0
        
        for data in dataloader:
            images, labels = data
            images = images.to(config["DEVICE"])
            labels = labels.to(config["DEVICE"])

            outputs = net(images)
            loss += criterion(outputs, labels)
            n += labels.size(0)
            
            _, predicted = torch.max(outputs.data, 1)
            correct += (predicted == labels).sum().item()

    net.train(True)
    return loss / n, 100 * correct / n

In [ ]:
from tqdm.notebook import tqdm
from statistics import mean

train_losses = []
test_losses, test_accuracies = [], []

accuracies = []

# running_loss = 0.0
for j in tqdm(range(config["EPOCH"])):
    epoch_loss, n = 0, 0

    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        inputs = inputs.to(config["DEVICE"])
        labels = labels.to(config["DEVICE"])

        optimizer.zero_grad()

        outputs = net(inputs)
        train_loss = criterion(outputs, labels)

        epoch_loss += train_loss
        n += labels.size(0)

        train_loss = train_loss / labels.size(0)
        train_loss.backward()
        optimizer.step()

    # compute the average training loss during this epoch
    epoch_loss = epoch_loss / n
    train_losses.append(epoch_loss)

    # Compute the average accuracy
    _, test_acc = compute_loss_accuracy(net, test_loader)
    # Average accuracy across the clients
    accuracies.append(test_acc)

    test_loss, test_acc = compute_loss_accuracy(net, test_loader)
    test_losses.append(test_loss)
    test_accuracies.append(test_acc)

    print(f"Epoch {j + 1:3d}: train_loss={epoch_loss:.4f}, test_loss={test_loss:.4f}, test_acc={test_acc:.2f}")

    scheduler.step(test_loss)

printJSON("Centralized-Model", accuracies, net)

In [ ]:
import shutil
shutil.make_archive('artifacts', 'zip', 'artifacts')

from IPython.display import FileLink
FileLink(r'artifacts.zip')

In [9]:
# Old JSON save
import time
import json
import os

timestr = time.strftime("%Y_%m_%d-%I_%M_%S_%p")
artifact_filename = f"artifacts/centralized-{timestr}"

if not os.path.exists("artifacts/"):
    os.mkdir("artifacts")

# parameters of the trained model
parameters = net.state_dict()
# save the model on the local file system
torch.save(parameters, artifact_filename + ".pth")

data = {
    "config": {k:v for k, v in config.items() if k != "DEVICE"},
    "train_losses": [float(e) for e in train_losses],
    "test_losses": [float(e) for e in test_losses],
    "test_accuracies": [float(e) for e in test_accuracies]
}

with open(artifact_filename + ".json", "w") as f:
    f.write(json.dumps(data, indent=4))